### SUCESSO DE ESTUDANTES

In [1]:
import pandas as pd
import numpy as np
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder, StandardScaler
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

%matplotlib inline

### IMPOTANDO BASE DE DADOS

In [2]:
estudantes = pd.read_csv("estudantes.csv", sep=";")
estudantes.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


### EXPLORANDO DADOS

In [ ]:
#informações dataset
estudantes.info()

In [ ]:
#número de linhas e coluans do dataframe
estudantes.shape

In [ ]:
#presença de nulos
estudantes.isnull().sum()

In [ ]:
# fig, ax = plt.subplots(1, sharey=True )
# heatmap = sns.heatmap(estudantes.select_dtypes(include="float64"), ax=ax)
# fig.set_size_inches(18.5, 10.5)


#### PREPROCESSAMENTO

In [ ]:
for i in range(37):
    estudantes = estudantes.rename({estudantes.columns[i]:i}, axis=1)


    

In [ ]:
labele = LabelEncoder()
for colunas in estudantes.iloc[:,:36].select_dtypes(include="object"):
    estudantes[colunas] = labele.fit_transform(estudantes.loc[colunas])

In [ ]:
#Redimensioando com a Padronização
padronizar = StandardScaler()
estudantes.iloc[:,:36] = padronizar.fit_transform(estudantes.iloc[:,:36])
estudantes.head()

In [ ]:
#Separando entre clase e previsores
previsores = estudantes.iloc[:, 0:36].values
classe =estudantes.iloc[:, 36].values
classe

In [ ]:
X_train, X_test, y_train,y_test = train_test_split(previsores, classe, test_size=0.3, random_state=42)

##### CRIANDO MODELOS

In [ ]:
clf = [ GaussianNB(), 
    DecisionTreeClassifier(random_state=42),  
    RandomForestClassifier(
    n_estimators=500, 
    min_samples_leaf=25, random_state=42), 

    GradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.01,
    subsample=1,
    max_depth=6,
    random_state=42
)]
print(len(clf))

In [ ]:

lista_clf=[]
for i in range(len(clf)):
    modelo = clf[i]
    modelo = modelo.fit(X_train, y_train)
    lista_clf.append(modelo)
    


In [ ]:
lista_prev =[]
for item in lista_clf:
    previsao = item.predict(X_test)
    lista_prev.append(previsao)
    

In [ ]:
lista_classificadores = ["Naive Bayes", "Árvore de Decisão","Floresta de Decisão", "XGBoost"]
contador=0
for item in lista_prev:
    acuracia = accuracy_score(y_test, item)
    mod = lista_classificadores[contador]
    print(f"{mod}: {round(acuracia,2)}")
    contador+=1